<a href="https://colab.research.google.com/github/Nyota-Lab/torch/blob/Modelo-de-reconocimiento-de-imagenes/Modelo_de_Clasificacion_de_Im%C3%A1genes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo de clasificacion de imagenes
* Vamos a entrenar un modelo de clasificacion de imagenes usando el dataset normalizado

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import utils
from torchvision import datasets
from torchvision import transforms

from PIL import Image
import urllib.request as request
import matplotlib.pyplot as plt

In [0]:
data_path = 'data/'
full_data_set = datasets.CIFAR10(data_path, train=True,download=True, transform=transforms.Compose([
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(
                                                          mean=[0.4915, 0.4823, 0.4468],
                                                          std=[0.2470, 0.2435, 0.2616]
                                                      )]))
full_validation_set = datasets.CIFAR10(data_path, train=False,download=True, transform=transforms.Compose([
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(
                                                          mean=[0.4915, 0.4823, 0.4468],
                                                          std=[0.2470, 0.2435, 0.2616]
                                                      )]))

* Vamos a generar un dataloader que nos va a permitir hacer batchs de imagenes de acuerdo a el tamaño que decidamos

In [0]:
size = 4
dataloader = torch.utils.data.DataLoader(full_data_set, batch_size=size, shuffle=True, num_workers=2)

* Una vez tenemos nuestra funcion dataloader vamos a aplicarla en nuestro dataset a través de un iterador

In [0]:
data_iterator = iter(dataloader)
imgs, labels = data_iterator.next()
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
plt.imshow(utils.make_grid(imgs).permute(1,2,0))
for i in range(size):
  print(classes[labels[i]])

* Al ser una clasificacion binaria vamos a tomar dos clases del dataset y solo nos vamos a quedar con los resultados que ingresen dentro de una de las clases reclasificando las etiquetas e instanciando dos nuevos sets de datos y entrenamiento que solo van a tener las imagenes con las clases seleccionanas

In [0]:
class_name = ['automobile', 'cat']
label_map = {1:0, 3:1}

data_set = [(img, label_map[label]) for img, label in full_data_set if label in [1,3]]
validation_set = [(img, label_map[label])for img, label in full_validation_set if label in [1,3]]

In [0]:
img, label = data_set[20]
img.shape

* Una vez tenemos nuestros datasets vamos a entrenar nuestro modelo.
  * Usamos un modelo lineal con valores de entrada de acuerdo al tamaño de nuestras imagenes y su dimension y nuestro valor de salida van a ser 512 'neuronas'
  * Nuestra capa oculta va a utilizar como funcion de activacion Tanh
  * De nuevo llamamos al modelo lineal pero esta vez nuestro valor de entrada van a ser las 512 'neuronas' y nuestro valor de salida va a ser 2 ya que nuestro modelo es binario
  * Agregamos la funcion Softmax debido a que queremos que nuestro resultado sea una probabilidad

In [0]:
model = nn.Sequential(
          nn.Linear(3*32*32,512),
          nn.Tanh(),
          nn.Linear(512,2),
          nn.Softmax(dim=1)
)

In [0]:
img, label = data_set[30]
plt.imshow(img.permute(1,2,0))
class_name[label]

In [0]:
out = model(img.view(-1).unsqueeze(0))
out

In [0]:
_, index = torch.max(out, dim=1)
print( class_name[label], '->', 'model:', class_name[index])